In [1]:
# import libraries
import pandas as pd
import numpy as np
import plotly.express as px
from pycaret.classification import *

# read csv data
data = pd.read_csv('Leads.csv')
data.head()

ModuleNotFoundError: No module named 'pycaret'

In [ ]:
# check data info
data.info()

In [ ]:
# creating a copy of data
data2 = data.copy()
data2.dropna(axis=0, inplace=True)
data2['Converted'] = data2['Converted'].replace({1 : 'Yes', 0 : 'No'})

# plotly visual

fig = px.scatter(x=data2['Total Time Spent on Website'], y=data2['Asymmetrique Activity Score'], 
                 color = data2['Converted'], template = 'plotly_white', 
                 facet_col = data2['Lead Origin'], facet_row = data2['Do Not Email'],
                 title = 'Lead Conversion by Total time spent on Website, Activity Score and Lead Origin',
                 labels = {'x' : 'Total Time Spent on Website', 'y' : 'Activity Score'})
                
fig.show()

In [ ]:
# init setup
s = setup(data, target = 'Converted', ignore_features = ['Prospect ID', 'Lead Number'])

In [ ]:
# compare all models
best_model = compare_models(sort='AUC')

In [ ]:
# print best_model parameters
print(best_model.get_all_params())

In [ ]:
# AUC Plot
plot_model(best_model, plot = 'auc')

In [ ]:
# Shapley Values
interpret_model(best_model)

In [ ]:
# Feature Importance
plot_model(best_model, plot = 'feature')

In [ ]:
# Confusion Matrix
plot_model(best_model, plot = 'confusion_matrix')

In [ ]:
# create a custom function
def calculate_profit(y, y_pred):
    tp = np.where((y_pred==1) & (y==1), (120-15), 0)
    fp = np.where((y_pred==1) & (y==0), -15, 0)
    fn = np.where((y_pred==0) & (y==1), -120, 0)
    return np.sum([tp,fp,fn])# add metric to PyCaret
add_metric('profit', 'Profit', calculate_profit)

In [ ]:
# compare all models
best_model = compare_models(sort='Profit')

In [ ]:
# confusion matrix
plot_model(best_model, plot = 'confusion_matrix')

In [ ]:
# create copy of data
data_new = data.copy()
data_new.drop('Converted', axis=1, inplace=True)# generate labels using predict_model
predict_model(best_model, data=data_new, raw_score=True)